# Black-Scholes Autodifferentiation Example
In this example we are calculating $\theta$ values using the Black-Scholes model via a Lazy Stream.

In [5]:
# Imports & Constants
import tributary.lazy as tl
import math
TRADING_DAYS = 252

## We can use the tributary $\textit{Erf}$ operator to define a Standard Normal CDF
The CDF is defined using $\Phi(x, \mu, \sigma) = \frac{1}{2}\left(1 + Erf\left(\frac{x-\mu}{\sigma\sqrt{2}}\right)\right)$ for $\mu = 0, \sigma = 1$

In [6]:
def normal_cdf(x):
    return ((tl.Erf(x / tl.Node((math.sqrt(2),0), use_dual=True)) + tl.Node((1,0), use_dual=True))
            / tl.Node((2,0), use_dual=True))

## Now we can define our stream
For this example we are going use a Call Option.
We define the model as follows:

$C_p = S\Phi(d_1) - Xe^{-rt}\Phi(d_2)$

$d_1 = \frac{ln\left(\frac{S}{X}\right) + \left(r + \frac{\sigma^2}{2}\right)t}{\sigma\sqrt{t}}$

$d_2 = d_1 - \sigma\sqrt{t}$

Where

$C_p$ - Price of Call Option

$S$ - Stock Price

$X$ - Strike Price

$r$ - Risk Free Interest Rate

$\sigma$ - Stock Price Volatility

$t$ - Time to Maturity

$\Phi$ - Standard Normal CDF (defined above)

### Lazy Graph

In [7]:
strike_price = tl.Node(value=(203, 0), name='Strike Price', use_dual=True)
stock_price = tl.Node(value=(210, 0), name='Stock Price', use_dual=True)
r = tl.Node(value=(0.2175, 0), name='Risk Free Interest Rate', use_dual=True)
time = tl.Div(
              tl.Node(value=(4, 1), name='Time to Maturity', use_dual=True), 
              tl.Node(value=(TRADING_DAYS,0), use_dual=True)
        )
vol = tl.Node(value=(14.04, 0), name='Stock Price Volatility', use_dual=True)

d1 = ((tl.Log(stock_price / strike_price) + time * (r + (vol**2 / tl.Node((2,0), use_dual=True)))) 
      / (vol * tl.Sqrt(time)))
    

d2 = d1 - vol * tl.Sqrt(time)

opt_price_lazy = stock_price * normal_cdf(d1)- strike_price *tl.Exp(tl.Negate(r * time)) * normal_cdf(d2)      

In [8]:
# Run it
opt_price_lazy()

(132.41454517196362, 12.327192918542838)